# Scraping and Cleaning Data from Wikipedia Pages for Netflix Engagement Analysis

### Data Sources:
**TV Shows**:
- [List of Netflix original programming](https://en.wikipedia.org/wiki/List_of_Netflix_original_programming)
- [List of Ended Netflix original programming](https://en.wikipedia.org/wiki/List_of_ended_Netflix_original_programming)

**Films**:
- [Lists of Netflix original films](https://en.wikipedia.org/wiki/Lists_of_Netflix_original_films) 

**Stand-up Comedy**:
- [List of Netflix stand-up comedy specials](https://en.wikipedia.org/wiki/List_of_Netflix_original_stand-up_comedy_specials#Series/collections)

### About the Data:
WIP


### Objective:
Here, we're creating a dataset of **released** Netflix original titles, excluding any upcoming programming. This cleaned dataset will help further my analysis of Netflix's ** viewer engagement patterns** from their 2024 Engagement Report.

By focusing on titles that are Netflix originals, we can gain deeper insights into:
- **Viewer engagement metrics**, such as watch time and popularity trends.
- **Content performance**, identifying patterns in the success or failure of various titles.
- **Genre or regional analysis**, allowing us to explore which types of content resonate most with Netflix's audience.

Leveraging the following libraries...
- *requests* - to efficiently retrieve raw HTML content from the Wikipedia pages
- *BeautifulSoup* - to parse the HTML content, search for the tables with original Netflix titles, and extract its content in an organized manner.


In [103]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [104]:

def scrape(url, type):
    # Send request to URL and parse HTML content
    try:
        data = requests.get(url).text
        soup = BeautifulSoup(data, 'html.parser')
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from {url}: {e}")
        return [], [], [], [], []
    
    # Initialize empty lists to store extracted data  
    titles, genres, languages, dates, content_type = [], [], [], [], []
  
    # Find all tables with the 'wikitable sortable' class
    tables = soup.find_all('table', {'class': 'wikitable sortable'})

    # Loop pulled tables 
    for table in tables:
        
        # Retrieve heading text (h2) of the current table 
        h2 = table.find_previous('h2').get_text(strip=True) 
   
        # Skip specific tables, all contain unreleased/undated titles
        skip_headings = ["Upcoming original programming", "Upcoming", "Undated films"]
        if h2 in skip_headings:
                continue      
        
        # Find column headers to find title's Release Date
        t_headers = table.find_all('th')
        for i, header in enumerate(t_headers):
                title = header.get_text(strip=True)
                if title == "Release date" or title == "Premiere":
                        date_index = i

        # Loop through each row in table, skip header row             
        rows = table.find_all('tr')
        for row in rows[1:]:
                cells =  row.find_all('td')
                if len(cells) > 0:
                        # Extract titles text and add to list
                        title = cells[0].get_text(separator=" ",strip=True)
                        titles.append(title)

                        # Extract release date using index number previously found
                        dates.append(cells[date_index].get_text(strip=True))

                        # Handle extraction process based on content type
                        if type == "TV":
                                content_type.append('TV')

                                # Within TV pages, handle extraction based on h2 text
                                if h2 == "Animation":
                                        languages.append(cells[-2].get_text(separator= " ", strip=True))
                                        
                                        # Extract genre from heading text (h3) in Animation section 
                                        h3 = table.find_previous('h3').string
                                        if h3 == None:
                                                genres.append("Kids & Family")

                                        elif h3 == "Adult animation":
                                                genre = cells[1].get_text(separator = " ", strip = True)
                                                genres.append(genre + " Adult animation")
                                                
                                        elif h3 == "Anime":
                                                genre = cells[1].get_text(separator = " ", strip = True)
                                                genres.append(genre + " Anime")

                                elif h2 == "Non-English language scripted":
                                        # Extract language from heading text (h3)
                                        h3 = table.find_previous('h3').string
                                        languages.append(h3)
                                        genres.append(cells[1].get_text(separator = " ", strip = True))

                                elif h2 == "Regional original programming":
                                        # Extract genre or language from h3 text
                                        h3 = table.find_previous('h3').string
                                        if h3 == "Animation":
                                                genres.append(h3)  
                                                languages.append(cells[-3].get_text(separator = " ", strip=True))     
                                        else:
                                                genres.append(cells[1].get_text(separator = " ", strip=True))
                                                if h3 == "Non-English language scripted":
                                                        languages.append("Spanish")
                                                        
                                                elif h3 == "Drama" or h3 == "Comedy":
                                                        languages.append("English")
                                                else:
                                                        languages.append(cells[-3].get_text(separator = " ", strip=True))                             
                                
                                elif h2 == "Continuations":
                                        genres.append(cells[1].get_text(separator = " ", strip = True))
                                        languages.append(cells[-2].get_text(separator = " ", strip = True))

                                elif h2 == 'Unscripted':
                                        genre = cells[1].get_text(separator = " ", strip = True)
                                        languages.append(cells[-2].get_text(separator = " ", strip = True))

                                        h3 = table.find_previous('h3').string
                                        if h3 == "Docuseries":
                                                genres.append(f"{genre}, Docuseries")
                                        elif h3 == "Variety":
                                                genres.append(genre)
                                        elif h3 == "Reality":
                                                genres.append("Reality")  

                                # General case for a TV Show table      
                                else: 
                                        genre = cells[1].get_text(separator = " ", strip = True)
                                        if h2 == "Specials":
                                                genres.append(f"{genre}, Specials")
                                                languages.append(cells[-2].get_text(separator = " ", strip = True))
                                                
                                        elif h2 == None:
                                                genres.append(f"{genre}, Kids & Family")
                                                languages.append("English")
                                        else:                
                                                genres.append(genre)
                                                languages.append("English")
                                  
                        elif type == "Film":
                                content_type.append("Film")
                                if h2 == "Documentaries":
                                        genres.append("Documentary")
                                        languages.append(cells[-1].get_text(strip = True))
                                else: 
                                        genre = (cells[2].get_text(separator = " ", strip = "True"))
                                        # Identify and handle genre cells containing misplace runtime info instead of genre
                                        if " min" in genre or genre == "2 h":
                                                genres.append(cells[1].get_text(separator=" ", strip=True))
                                        else: 
                                                genres.append(genre)

                                        language = cells[-1].get_text(separator=" ", strip=True)
                                        languages.append(language)
                        
                        elif type == "Standup":
                                content_type.append("TV")    
                                genres.append('Stand-up, Comedy')

                                if h2 == 'Series/collections':
                                        languages.append(cells[-2].get_text(separator=" ", strip=True))
                                else:
                                        languages.append(cells[-1].get_text(separator=" ", strip=True))                                                             
        
    return titles, genres, languages, dates, content_type

In [105]:
# Define URLs by type  
TV_urls = [
    "https://en.wikipedia.org/wiki/List_of_Netflix_original_programming",
    "https://en.wikipedia.org/wiki/List_of_ended_Netflix_original_programming"
]
Film_urls = [
    "https://en.wikipedia.org/wiki/List_of_Netflix_original_films_(2015%E2%80%932017)",
    "https://en.wikipedia.org/wiki/List_of_Netflix_original_films_(2018)",
    "https://en.wikipedia.org/wiki/List_of_Netflix_original_films_(2019)",
    "https://en.wikipedia.org/wiki/List_of_Netflix_original_films_(2020)",
    "https://en.wikipedia.org/wiki/List_of_Netflix_original_films_(2021)",
    "https://en.wikipedia.org/wiki/List_of_Netflix_original_films_(2022)",
    "https://en.wikipedia.org/wiki/List_of_Netflix_original_films_(2023)",
    "https://en.wikipedia.org/wiki/List_of_Netflix_original_films_(since_2024)"
]
S_url = ["https://en.wikipedia.org/wiki/List_of_Netflix_original_stand-up_comedy_specials"]

# Initialize empty lists to store all extracted data across all URLs
all_titles = []
all_genres = []
all_languages = []
all_dates = []
all_types = []

urls_list = [TV_urls, Film_urls,S_url]
type_list = ["TV","Film","Standup"]

for i in range(3):
        for url in urls_list[i]:
                titles, genres, languages, dates, types = scrape(url, type_list[i])
                all_titles.extend(titles)
                all_genres.extend(genres)
                all_languages.extend(languages)
                all_dates.extend(dates)
                all_types.extend(types)


original_titles_df = pd.DataFrame({
    'Title': all_titles,
    'Season':  pd.NA,
    'Genres': all_genres,
    'Language': all_languages,
    'Release Date': all_dates,
    'Content Type': all_types
    })  
  
original_titles_df.head()

,Title,Season,Genres,Language,Release Date,Content Type
0,Stranger Things,<NA>,Science fiction horror,English,"July 15, 2016",TV
1,Virgin River,<NA>,Romantic drama,English,"December 6, 2019",TV
2,The Witcher,<NA>,Fantasy drama,English,"December 20, 2019",TV
3,Outer Banks,<NA>,Coming-of-age drama,English,"April 15, 2020",TV
4,Sweet Magnolias,<NA>,Romantic drama,English,"May 19, 2020",TV


In [106]:
original_titles_df['Genres'].unique()

array(['Science fiction horror', 'Romantic drama', 'Fantasy drama',
       'Coming-of-age drama', 'Teen drama',
       'Alternate historical romance', 'Drama', 'Legal thriller',
       'Fantasy', 'Horror', 'Spy adventure', 'Political thriller',
       'Science fiction', 'Superhero drama', 'Conspiracy thriller',
       'Thriller', 'Thriller drama', 'Western', 'Comedy drama', 'Sitcom',
       'Comedy', 'Comedy horror', 'Comedy drama anthology',
       'Romance comedy drama', 'Spy action comedy', 'Action comedy',
       'Thriller comedy drama', 'Workplace comedy', 'Crime comedy',
       'Dark comedy', 'Action-adventure',
       'Coming-of-age comedy Adult animation',
       'Anthology Adult animation',
       'Science fiction comedy Adult animation', 'Action Adult animation',
       'Action-adventure Adult animation', 'Sitcom Adult animation',
       'Comedy Adult animation', 'Fantasy Adult animation',
       'Fantasy action Anime', 'Dark fantasy Anime',
       'Fantasy adventure Anime', 

## Creating a more refined and readable 'Genres' column:

In [107]:
genre_keywords = {
    'Anime': {'anime'},
    'Adult Animation': {'adult animation'},
    'Animation': {'animation', 'stop motion'},
    'Action': {'action', 'adventure', 'spy', 'war', 'martial arts', 'disaster', 'superhero', 'survival'},
    'Anthology': {'anthology'},
    'Biography': {'biopic', 'biography'},
    'History': {'history','period', 'historical'},
    'Horror': {'horror', 'slasher'},
    'Science Fiction': {'sci-fi', 'science fiction', 'zombie', 'mecha', 'kaiju', 'apocalyptic', 'dystopian'},
    'Fantasy': {'fantasy', 'supernatural','space', 'monster'},
    'Comedy': {'comedy', 'humor', 'mockumentary','dramedy', 'sitcom', 'satire'},
    'Drama': {'drama', 'biopic', 'dramedy', 'western','telenovela', 'historical'},
    'Documentary': {'documentary', 'docuseries'},
    'Romance': {'romance', 'romantic'},
    'Reality': {'reality','competition','dating show', 'prank', 'competitive', 'quiz','panel', 'cooking show'},
    'Sports': {'sport'},
    'Thriller': {'thriller'},
    'Mystery': {'mystery', 'detective'},
    'Stand-Up': {'stand-up', 'one-man show', 'speech'},
    'Talk Show': {'talk show', 'interview', 'late-night'},
    'Teen': {'teen','coming-of-age', 'coming of age'},
    'Kids & Family': {'kids & family', 'children', 'family', 'preschool', 'educational','education'},
    'Crime': {'crime','heist', 'drug', 'police'},
    'Musical': {'musical'},
    'Specials': {'specials', 'behind the scenes', 'award show', 'event', "director's cut", 'concert', 'variety', 'making', 'clipshow', 'aftershow', 'fitness'}
}

def categorize_genres(genre):
    # Convert from a list to a string in order to make it lowercase
    genre = "".join(genre)
    genre = genre.lower()

    matched = []

    # Check for keyword in each genre category
    for main_genre, keywords in genre_keywords.items():
        # If found in the genre text, add to matched list
        if any(keyword in genre for keyword in keywords):
            matched.append(main_genre)

    
    # Join matched genres, or return original genre text
    return ','.join(matched) if matched else genre

# Apply function to the 'Genres' column
original_titles_df['Genres'] = original_titles_df['Genres'].apply(categorize_genres)

In [108]:
# Function to find unwanted/irrelevant genres
def genre_checker():
    # Create set to collect unique genres
    unique_genres = set()

    # Split each genre string by commas and add each genre to the set
    for genres in original_titles_df['Genres']:
        genre_list = [genre for genre in genres.split(',')]
        unique_genres.update(genre_list)

     # Remove genres that match any of the main categories set earlier 
    for genre in genre_keywords.keys():
        if genre in unique_genres:
            unique_genres.discard(genre)
        
    unique_genres_list = (unique_genres)

    # Display the unique genres list
    return (unique_genres_list)

print(genre_checker())

{'christmas film', 'holiday', 'buddy cop'}


In [109]:
targets = []
index = 0
for genres in original_titles_df['Genres']:
    if any(keyword in genres for keyword in ['drugs', 'police procedural', 'christmas film', 'buddy cop']):
        targets.append(original_titles_df.iloc[index]['Title'])
    index += 1
print(targets)

['1000 Miles from Christmas', 'Beverly Hills Cop: Axel F']


In [110]:
original_titles_df[original_titles_df['Title'].str.contains('Feed')]

,Title,Season,Genres,Language,Release Date,Content Type
208,Somebody Feed Phil,<NA>,Documentary,English,"January 12, 2018",TV
1024,Feedback,<NA>,Drama,Polish,"November 15, 2023",TV


In [111]:
# Map titles to their proper genres after researching each
title_new_genres = {
    'Inside the Real Narcos': 'Documentary,Crime',
    '1000 Miles from Christmas': 'Romance,Comedy',
    'Beverly Hills Cop: Axel F': 'Action,Comedy'
}

for i, title in enumerate(original_titles_df['Title']):
    # Update genres for each title above
    if title in title_new_genres.keys():
        original_titles_df.loc[i, 'Genres'] = title_new_genres[title]

if(len(genre_checker()) == 0):
    print("All good to go!")

## Handling titles and dates with irrelevant [...] section
Removing unnecessary characters from `Titles` and `Release Date` columns.

In [112]:
# Closer look at these [] characters
original_titles_df[original_titles_df['Title'].str.contains(r'\[.*?\]', regex=True)].head(3)

,Title,Season,Genres,Language,Release Date,Content Type
22,The Madness [ 25 ] [ 26 ],<NA>,Thriller,English,"November 28, 2024[27]",TV
23,Black Doves [ 28 ] [ 29 ],<NA>,Thriller,English,"December 5, 2024[30]",TV
24,Missing You [ 32 ] [ 33 ],<NA>,"Drama,Thriller",English,"January 1, 2025[34]",TV


After searching through the Netflix Engagement Report csv, there is only one film with these [] characters in the title: *[REC]*. As we can see below, this film isn't in our dataframe, so we can deduce that *[REC]* is not a Netflix Original. 

In [113]:
# Search for 'REC' in titles
original_titles_df[original_titles_df['Title'].str.contains('REC')]

,Title,Season,Genres,Language,Release Date,Content Type


Let's move forward with removing all traces of [] characters, and its contents, from the `Title` and `Release Date` column.

In [114]:
# Remove [], any content inside the brackets, and strip whitespace 
original_titles_df['Title'] = original_titles_df['Title'].str.replace(r'\[.*?\]', '', regex=True)  
original_titles_df['Title'] = original_titles_df['Title'].str.strip()

# Check changes
original_titles_df[original_titles_df['Title'].str.contains(r'\[.*?\]', regex=True)]

,Title,Season,Genres,Language,Release Date,Content Type


In [115]:
# Repeating steps for Release Date column 
original_titles_df[original_titles_df['Release Date'].str.contains(r'\[.*?\]', regex=True)].head(3)

,Title,Season,Genres,Language,Release Date,Content Type
22,The Madness,<NA>,Thriller,English,"November 28, 2024[27]",TV
23,Black Doves,<NA>,Thriller,English,"December 5, 2024[30]",TV
24,Missing You,<NA>,"Drama,Thriller",English,"January 1, 2025[34]",TV


In [116]:
# Remove [] + any content inside the brackets, and strip whitespace 
original_titles_df['Release Date'] = original_titles_df['Release Date'].str.replace(r'\[.*?\]', '', regex=True)  
original_titles_df['Release Date'] = original_titles_df['Release Date'].str.strip()

# Check changes
original_titles_df[original_titles_df['Release Date'].str.contains(r'\[.*?\]', regex=True)]

,Title,Season,Genres,Language,Release Date,Content Type


## Handling titles released after the report date
Ensuring that no titles included were released after Netflix's latest Engagement Report, dated September 19, 2024.

In [117]:
# Convert 'Release Date' column to datetime
original_titles_df['Release Date'] = pd.to_datetime(original_titles_df['Release Date'], errors='coerce')

print("Some Rows with 'Release Date' on or after September 19, 2024:")
original_titles_df[original_titles_df['Release Date'] >= "September 19, 2024"].head()

Some Rows with 'Release Date' on or after September 19, 2024:


,Title,Season,Genres,Language,Release Date,Content Type
20,Beauty in Black,<NA>,Drama,English,2024-10-24,TV
21,Territory,<NA>,Drama,English,2024-10-24,TV
22,The Madness,<NA>,Thriller,English,2024-11-28,TV
23,Black Doves,<NA>,Thriller,English,2024-12-05,TV
24,Missing You,<NA>,"Drama,Thriller",English,2025-01-01,TV


In [118]:
# Disinclude titles that were released on or after the report's release
original_titles_df = original_titles_df[original_titles_df['Release Date'] < "September 19, 2024"]

## Handling titles with multiple seasons/segments
In Netflix's Engagement Report, each season of a title is listed separately. For example, rather than displaying _Black Mirror (series 3-6)_ as a single entity, the report has an individual entry for each season, like so:
 - Black Mirror (series 3)
 - Black Mirror (series 4)
 - Black Mirror (series 5)
 - Black Mirror (series 6)

To simplify later matching and comparison with Netflix's report, I am organizing each season as an individual entry as well.

In [119]:
# Define regex pattern that will match titles with a single '(' followed by one word, a space, digits, a dash, more digits, and ')'
# This pattern will match sequences like '(series 1–3)' or '(seasons 3–6)'
pattern = r'\((\w+)\s(\d+)–(\d+)\)'

# Filter dataframe to find titles that contain this pattern
original_titles_df[original_titles_df['Title'].str.contains(pattern)]['Title']

/var/folders/dt/npwm56kx383b7_d7m4gcvwch0000gn/T/ipykernel_53316/2466735749.py:6: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  original_titles_df[original_titles_df['Title'].str.contains(pattern)]['Title']


331                             Black Mirror (series 3–6)
332                                     You (seasons 2–4)
333                    Unsolved Mysteries (seasons 15–17)
334                               Cobra Kai (seasons 3–6)
335                              Blown Away (seasons 2–4)
336                            Kota Factory (seasons 2–3)
337                                The Mole (seasons 6–7)
342                            Bad Exorcist (seasons 3–5)
348     The Parisian Agency: Exclusive Properties (sea...
351                             Skam Italia (seasons 4–6)
352                              CoComelon (seasons 6–11)
638                            Bad Exorcist (seasons 1–2)
1678                     Trailer Park Boys (seasons 8–12)
1679                               Lovesick (seasons 2–3)
1680                       The Staircase (episodes 11–13)
1681                          Paquita Salas (seasons 2–3)
1682     Comedians in Cars Getting Coffee (seasons 10–11)
1683    Inside

As shown above, there are four kinds of segments: seasons, series, episodes, and parts. Let's take a closer look at episodes and parts since they occur less.

In [120]:
# Filter dataframe to find titles that contain either '(episodes' or '(parts'
original_titles_df[original_titles_df['Title'].str.contains(r'(\(episodes|parts)')]

/var/folders/dt/npwm56kx383b7_d7m4gcvwch0000gn/T/ipykernel_53316/1734951219.py:2: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  original_titles_df[original_titles_df['Title'].str.contains(r'(\(episodes|parts)')]


,Title,Season,Genres,Language,Release Date,Content Type
1680,The Staircase (episodes 11–13),<NA>,Documentary,English,2018-06-08,TV
1691,Money Heist (parts 3–5),<NA>,"Drama,Crime",Spanish,2019-07-19,TV


- Netflix’s Engagement Report lists *The Staircase* as a complete series rather than by individual episodes. Since Netflix only produced three episodes, I am removing this title from `original_titles_df` because the entire show should not be considered a Netflix Original.

- *Money Heist* will be handled just like titles with seasons/series.



In [121]:
# dropping 'The Staircase' from dataframe
original_titles_df = original_titles_df[original_titles_df['Title'] != 'The Staircase (episodes 11–13)']

# Identify titles containing previously set pattern
# pattern matches sequences like '(series 1-2)' / '(seasons 3-6)' / '(parts 3-5)'
titles_multiple_seasons = original_titles_df[original_titles_df['Title'].str.contains(pattern)]['Title']

print(f'There are {len(titles_multiple_seasons)} titles with multiple seasons/series')
titles_multiple_seasons.head()

There are 34 titles with multiple seasons/series


/var/folders/dt/npwm56kx383b7_d7m4gcvwch0000gn/T/ipykernel_53316/1972366276.py:6: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  titles_multiple_seasons = original_titles_df[original_titles_df['Title'].str.contains(pattern)]['Title']


331             Black Mirror (series 3–6)
332                     You (seasons 2–4)
333    Unsolved Mysteries (seasons 15–17)
334               Cobra Kai (seasons 3–6)
335              Blown Away (seasons 2–4)
Name: Title, dtype: object

In [122]:
import re 

# Initialize lists to collect newly created rows and the indices of rows to drop
new_rows = []
indices_to_drop = []

# Check if title contains text in this format '(abc xx-xx)'
# It will match sequences like '(series 1–3)' or '(seasons 3–6)'

# Iterate over df 
for index, row in original_titles_df.iterrows():
    title = row['Title']

    pattern = r'\s\((\w+)\s(\d+)–(\d+)\)'

    match = re.search(pattern,title)
    if match:
        segment = match.group(1)                # Captures first word inside ()
        start_season = int(match.group(2))      # First digit(s)
        end_season = int(match.group(3))        # Last digit(s)
        
        for season_num in range(start_season, end_season + 1):
            # In the report, Money Heist is separated by parts
            if segment == 'parts':          
                new_title = re.sub(pattern, '', title)
            # In the report, all Netflix Originals seem to be separated by seasons
            else:   
                new_title = re.sub(pattern, '', title)
            
            # Only have the release date for a show's start season
            date = row['Release Date'] if season_num == start_season else pd.NaT     
                
            new_row = {
                'Title': new_title, 
                'Season': season_num,
                'Genres': row['Genres'], 
                'Language': row['Language'], 
                'Release Date': date,
                'Content Type': row['Content Type']
            }

            new_rows.append(new_row)  
            
        # Once done eliminate row with multiple seasons/series in the title from our dataframe
        indices_to_drop.append(index)

# Drop the rows with multiple seasons/series titles
print(f'There are {len(titles_multiple_seasons)} titles with multiple seasons/series/parts to drop')
original_titles_df = original_titles_df.drop(indices_to_drop).reset_index(drop=True)

# Confirm no rows with multi-season titles exist
final_check = original_titles_df[original_titles_df['Title'].str.contains(pattern, regex=True)]['Title']
if len(final_check) > 0:
    print("Some rows with multiple seasons/series remain:\n", final_check)
else:
    print("All necessary rows deleted, we're good to go!\n")  

# Convert list of dictionaries into a dataframe and concatenate it with the one we've been using 
new_rows_df = pd.DataFrame(new_rows)
original_titles_df = pd.concat([original_titles_df, new_rows_df], ignore_index=True)

There are 34 titles with multiple seasons/series/parts to drop
All necessary rows deleted, we're good to go!



/var/folders/dt/npwm56kx383b7_d7m4gcvwch0000gn/T/ipykernel_53316/577721625.py:52: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  final_check = original_titles_df[original_titles_df['Title'].str.contains(pattern, regex=True)]['Title']


In [123]:
print(original_titles_df[original_titles_df['Title'].str.contains('Black Mirror')][['Title','Season']])
print(original_titles_df[original_titles_df['Title'].str.contains('Money Heist')][['Title', 'Season']])

                           Title Season
1626  Black Mirror: Bandersnatch   <NA>
3500                Black Mirror      3
3501                Black Mirror      4
3502                Black Mirror      5
3503                Black Mirror      6
                                         Title Season
895   Money Heist: Korea – Joint Economic Area   <NA>
1653               Money Heist: The Phenomenon   <NA>
1762        Money Heist : From Tokyo to Berlin   <NA>
3568                               Money Heist      3
3569                               Money Heist      4
3570                               Money Heist      5


In [124]:
pattern = r'\s\(season\s(\d)\)'

for index,row in original_titles_df.iterrows():
    title = row['Title']
    match = re.search(pattern, title)
    
    if match:
        season_num = int(match.group(1))
        title = re.sub(pattern, '', title)
        original_titles_df.at[index, 'Title'] = title
        original_titles_df.at[index, 'Season'] = season_num    

## Removing spaces before colon
All English titles in the Netflix's report don't have this space, so just to make the cross analysis easier, I'm removing spaces before colons in `netflix_originals_df`

In [125]:
original_titles_df[original_titles_df['Title'].str.contains(r'\s+:\s+')].head()

,Title,Season,Genres,Language,Release Date,Content Type
65,Little Baby Bum : Music Time,<NA>,Kids & Family,English,2023-09-25,TV
177,World War II : From the Frontlines,<NA>,"History,Documentary",English,2023-12-07,TV
183,Alexander : The Making of a God,<NA>,"History,Documentary",English,2024-01-31,TV
187,Turning Point : The Bomb and the Cold War,<NA>,"History,Documentary",English,2024-03-12,TV
196,Hitler and the Nazis : Evil on Trial,<NA>,"History,Documentary",English,2024-06-05,TV


In [126]:
original_titles_df['Title'] = original_titles_df['Title'].str.replace(r' : ',': ')
original_titles_df[original_titles_df['Title'].str.contains(r'\s+:\s+')]

,Title,Season,Genres,Language,Release Date,Content Type


In [127]:
original_titles_df[original_titles_df['Title'].str.contains(r'.*?:\s(Season|Series|Part)\s(\d+|\w+)')]

/var/folders/dt/npwm56kx383b7_d7m4gcvwch0000gn/T/ipykernel_53316/658333493.py:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  original_titles_df[original_titles_df['Title'].str.contains(r'.*?:\s(Season|Series|Part)\s(\d+|\w+)')]


,Title,Season,Genres,Language,Release Date,Content Type
1709,Making The Witcher: Season 2,<NA>,Specials,English,2021-12-17,TV
1733,Making The Witcher: Season 3,<NA>,Specials,English,2023-07-27,TV
2281,The Christmas Chronicles: Part Two,<NA>,Comedy,English,2020-11-25,Film
2989,Code 8: Part II,<NA>,Science Fiction,English,2024-02-28,Film


In [128]:
season_keywords = {
    '2': 2,
    '3': 3,
    'Two': 2,
    'II': 2
}

for index, row in original_titles_df.iterrows():
    title = row['Title']
    pattern = r':\s(Season|Part)\s(\d|\w+)'
    if row['Content Type'] =='TV':
        match = re.search(pattern,title)
        if match:
            title = re.sub(pattern, '', title)
            original_titles_df.at[index, 'Title'] = title

            season_num = match.group(2)
            
            if season_num in season_keywords.keys():
                original_titles_df.at[index, 'Season'] = season_keywords[season_num]



In [129]:
original_titles_df['Season'] = original_titles_df['Season'].fillna('ALL')
original_titles_df['Season'].unique()

array(['ALL', 3, 4, 2, 6, 5, 15, 16, 17, 7, 8, 9, 10, 11, 1, 12, 29, 30],
      dtype=object)

In [130]:
original_titles_df[original_titles_df['Season']=='ALL']

,Title,Season,Genres,Language,Release Date,Content Type
0,Stranger Things,ALL,"Horror,Science Fiction",English,2016-07-15,TV
1,Virgin River,ALL,"Drama,Romance",English,2019-12-06,TV
2,The Witcher,ALL,"Fantasy,Drama",English,2019-12-20,TV
3,Outer Banks,ALL,"Drama,Teen",English,2020-04-15,TV
4,Sweet Magnolias,ALL,"Drama,Romance",English,2020-05-19,TV
...,...,...,...,...,...,...
3495,Only Jokes Allowed,ALL,"Comedy,Stand-Up",English,2022-02-09,TV
3496,That's My Time with David Letterman,ALL,"Comedy,Stand-Up",English,2022-06-07,TV
3497,Romesh Ranganathan: The Cynic,ALL,"Comedy,Stand-Up",English,2022-11-29,TV
3498,Michelle Wolf: It's Great To Be Here,ALL,"Comedy,Stand-Up",English,2023-09-12,TV


In [136]:
original_titles_df.to_csv('NetflixOriginalsReleaseDatesNA.csv', index=False)